In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import datetime
import timedelta

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# 현재 날짜 가져오기 , format = YYYYMMDD
today_iso = datetime.datetime.now().isoformat()[:10].replace('-','')

# 현재 날짜 가져오기 , format = YYYY-MM-DD
today = datetime.datetime.now()
today_s = str(today.year) + '-' +  str(today.month) + '-' +str(today.day)

# 19970101 ~ 현재 갯수 구하기
tot_len = len(pd.date_range("1997-5-4", today_s))

In [3]:
# 기준금리 데이터 가져오기 
# 한국은행 API 이용한 한국은행 기준금리 불러오기

from urllib.request import urlopen
import json

key = '❌'

url="http://ecos.bok.or.kr/api/StatisticSearch/" + key + "/json/kr/1/" + str(tot_len) + "/"  + "098Y001" + "/DD/19970101/" + today_iso + "/" + "0101000" +"/?/?/"


result = urlopen(url)
html = result.read()
data = json.loads(html)
data = data["StatisticSearch"]["row"]
data = pd.DataFrame(data)
data["TIME"] = data["TIME"].astype(str)

#컬럼 정리 : 
# TIME => Date : YYYY-MM-DD
# DATA_VALU => kr_bs_rate
data["Date"] = data["TIME"].str[:4] + "-" + data["TIME"].str[4:6] + "-" + data["TIME"].str[6:]
data = data.rename(columns={'DATA_VALUE' : 'kr_bs_rate'})
data = data[["Date","kr_bs_rate"]]
data = data.reset_index(drop=True)

# 금리(한국은행 기준금리) CSV 저장
data.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ "kr_bs_rate"+".csv", index=False)
data

,Date,kr_bs_rate
0,1999-05-06,4.75
1,1999-05-07,4.75
2,1999-05-08,4.75
3,1999-05-09,4.75
4,1999-05-10,4.75
...,...,...
7700,2020-06-05,0.5
7701,2020-06-06,0.5
7702,2020-06-07,0.5
7703,2020-06-08,0.5


In [4]:
# 미국 금리 데이터
# CSV 파일 이후 시계열에 대해서 금리 변경없는 경우 같은 값으로 계속 대입

base_df = pd.read_csv("D:\\시세모니터링\\융합분석\\DATA\\미국금리/WEBSTATS_CBPOL_D_DATAFLOW_csv_row.csv")
base_rate_us = base_df[["Frequency","D:Daily.35"]]
base_rate_us = base_rate_us.drop(base_rate_us.index[0:2])
base_rate_us = base_rate_us.reset_index().drop(columns=["index"])
base_rate_us.columns = ["date","b_rate_us"]
base_rate_us = base_rate_us.dropna()
# base_rate_us = base_rate_us.append({'date' : '2019-11-29', 'b_rate_us' : 1.625},ignore_index=True)
base_rate_us["date"] = pd.to_datetime(base_rate_us["date"])
# base_rate_us = base_rate_us.drop(columns=["date"])
# base_rate_us["date"] =(base_rate_us["date"].astype(str).str[0:4] + base_rate_us["date"].astype(str).str[5:7] + base_rate_us["date"].astype(str).str[8:10]).astype(int)  
base_rate_us = base_rate_us.rename(columns={'b_rate_us' : 'us_bs_rate'})
base_rate_us = base_rate_us[["date", "us_bs_rate"]]
base_rate_us = base_rate_us.rename(columns={"date" : 'Date'})
base_rate_us["us_bs_rate"] = base_rate_us["us_bs_rate"].astype(float)
base_rate_us

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,us_bs_rate
3103,1954-07-01,1.130
3104,1954-07-02,1.250
3105,1954-07-03,1.250
3106,1954-07-04,1.250
3107,1954-07-05,0.880
...,...,...
27134,2020-04-16,0.125
27135,2020-04-17,0.125
27136,2020-04-18,0.125
27137,2020-04-19,0.125


In [5]:
import datetime
import timedelta
# dt = datetime.datetime.now()

# 오늘날짜 기준
dt_p = base_rate_us.iloc[-1]["Date"] + datetime.timedelta(days=1)
dt_p

date_all = pd.date_range(str(dt_p)[:4] + "-" + str(dt_p)[5:7] + "-" + str(dt_p)[8:10], datetime.datetime.now().isoformat()[:10])
date_all = date_all.to_frame().reset_index()
# date_all.columns = ["date"]
date_all = date_all.rename(columns={0 : "Date"})
date_all = date_all.drop(columns=["index"])
# date_all = date_all[date_all["date"].dt.day_name() == "Friday"]
date_all

,Date
0,2020-04-21
1,2020-04-22
2,2020-04-23
3,2020-04-24
4,2020-04-25
5,2020-04-26
6,2020-04-27
7,2020-04-28
8,2020-04-29
9,2020-04-30


In [6]:
us_bs_rate_ft = date_all
us_bs_rate_ft["us_bs_rate"] = base_rate_us.iloc[-1]["us_bs_rate"]
us_bs_rate_ft

,Date,us_bs_rate
0,2020-04-21,0.125
1,2020-04-22,0.125
2,2020-04-23,0.125
3,2020-04-24,0.125
4,2020-04-25,0.125
5,2020-04-26,0.125
6,2020-04-27,0.125
7,2020-04-28,0.125
8,2020-04-29,0.125
9,2020-04-30,0.125


In [7]:
# 금리(미국중앙은행 기준금리) CSV 저장
base_rate_us = pd.concat([base_rate_us, us_bs_rate_ft], axis=0)
# base_rate_us.to_csv("bank_rate_" + fl_dt + ".csv")
base_rate_us = base_rate_us.reset_index(drop=True)
base_rate_us.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ "us_bs_rate"+".csv", index= False)
base_rate_us

,Date,us_bs_rate
0,1954-07-01,1.130
1,1954-07-02,1.250
2,1954-07-03,1.250
3,1954-07-04,1.250
4,1954-07-05,0.880
...,...,...
24080,2020-06-08,0.125
24081,2020-06-09,0.125
24082,2020-06-10,0.125
24083,2020-06-11,0.125


In [8]:
base_rate_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24085 entries, 0 to 24084
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        24085 non-null  datetime64[ns]
 1   us_bs_rate  24085 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 376.5 KB
